# Simple GNN
Build a simple GNN model with PyTorch Geometric.
* Dataset: Cora
* Model: GCN

## Load dataset

In [2]:
from torch_geometric.datasets import Planetoid

# Load the Cora dataset
dataset = Planetoid(root='/tmp/Cora', name='Cora')
data = dataset[0]

Processing...
Done!


## Visualize dataset

In [3]:
# visualize the graph structure with networkx and matplotlib
from torch_geometric.utils import to_networkx
import networkx as nx
import matplotlib.pyplot as plt

data_nx = to_networkx(data)
plt.figure(figsize=(12,10))
nx.draw(data_nx, with_labels=True, node_color=data.y, cmap='tab20', node_size=10)
plt.show()

KeyboardInterrupt: 

## Build model

In [ ]:
# Define a simple GNN model
import torch
import torch.nn as nn
from torch_geometric.nn import GCNConv

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(dataset.num_features, 32)
        self.relu = nn.ReLU()
        self.conv2 = GCNConv(32, dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = self.relu(x)
        x = self.conv2(x, edge_index)
        return x

net = Net()
net.cuda() # run on GPU

## Train model

In [ ]:
# define the loss function and the optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters())

data = data.cuda() # run on GPU

# train the model
net.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = net(data)
    loss = loss_fn(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

## Evaluate model

In [ ]:
# evaluate the model
net.eval()
pred = net(data).argmax(dim=1)
correct = pred[data.test_mask].eq(data.y[data.test_mask]).sum().item()
acc = correct / data.test_mask.sum().item()
print('Accuracy: {:.4f}'.format(acc))